# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [28]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [6]:
driver = webdriver.Firefox()

In [7]:
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [8]:
# Go to disclaimer and click stuff
driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')
driver.find_element_by_id('checkbox').click()
driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form').submit()

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [23]:
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [24]:
Select(driver.find_element_by_id(
  'slcJurisdiction')).select_by_visible_text('Statewide')

In [25]:
driver.find_element_by_id('txtTradeName').send_keys('VAP%')

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [26]:
driver.find_element_by_name('searchForm').submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [15]:
results_table = driver.find_element_by_css_selector(
  'table[summary="Search Results '
  'Table"]')
titles = results_table.find_elements_by_css_selector(
  'tr.searchfieldtitle > td.searchlistitem > span.copybold')

for title in titles:
  print(title.text)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [17]:
link_list = []

for title_element in results_table.find_elements_by_css_selector(
    'tr.searchfieldtitle'):
  try:
    href = title_element.find_element_by_css_selector('td.searchlistitem + '
                                                       'td > '
                                               'a').get_attribute('href')
    print(href)
    link_list.append(href)
  except:
    print('sorry, no link here')


https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ty00ak8Tdnk%3D
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=AGAXVi3VweA%3D
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=WqxNSeBUqqM%3D
sorry, no link here
sorry, no link here


Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [20]:
for link in link_list:
  driver.get(link)
  data = {
    'mailing_address': driver.find_element_by_xpath('/html/body/table['
                                                    '2]/tbody/tr[4]/td['
                                                    '2]/table[1]/tbody/tr['
                                                    '3]/td[1]').text,
    'location_address': driver.find_element_by_xpath('/html/body/table['
                                                     '2]/tbody/tr[4]/td['
                                                     '2]/table[1]/tbody/tr['
                                                     '5]/td').text,
    'licence_information': driver.find_element_by_xpath('/html/body/table['
                                                        '2]/tbody/tr[4]/td['
                                                        '2]/table['
                                                        '1]/tbody/tr[3]/td['
                                                        '2]').text,
    'total_amount_paid': driver.find_element_by_xpath('/html/body/table['
                                                      '2]/tbody/tr[4]/td['
                                                      '2]/table[2]/tbody/tr['
                                                      'last()]/td[last('
                                                      ')]').text,
    'issued_by': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr['
                                              '4]/td[2]/table[3]/tbody/tr['
                                              '2]/td').text
  }
  print(data)
  
driver.find_element_by_partial_link_text('Return to List').click()

{'mailing_address': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804', 'location_address': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801', 'licence_information': 'License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury', 'total_amount_paid': '$ 41.00', 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}


{'mailing_address': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804', 'location_address': 'VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801', 'licence_information': 'License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury', 'total_amount_paid': '$ 24.00', 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}


{'mailing_address': 'ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\nODENTON, MD 21113', 'location_address': 'VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054', 'licence_information': 'License Status: Issued\nLicense No.: 02304705\nControl No.: 02685930\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County', 'total_amount_paid': '$ 94.00', 'issued_by': 'ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401'}


In [22]:
# oh, wait, I probably wasn't supposed to do it like that there.
# well
# this is awkward
# guess I'm just too used to thinking like a programmer already

### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [38]:
url_list = []

results_table = driver.find_element_by_css_selector(
  'table[summary="Search Results '
  'Table"]')
rows = results_table.find_elements_by_css_selector(
  'tr.searchfieldtitle')

for row in rows:
  name = row.find_element_by_css_selector('td.searchlistitem > '
                                          'span.copybold').text

  try:
    link = row.find_element_by_css_selector('td.searchlistitem + td > '
                                            'a').get_attribute('href')
  except:
    link = ''

  data = {
    'name': name,
    'url': link
  }
  
  print(data)
  
  url_list.append(data)

df = pd.DataFrame(url_list)
df

{'name': 'VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ty00ak8Tdnk%3D'}
{'name': 'VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=AGAXVi3VweA%3D'}
{'name': 'VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=WqxNSeBUqqM%3D'}
{'name': 'VAPE FROG', 'url': ''}
{'name': 'VAPE FROG', 'url': ''}


,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,
4,VAPE FROG,


### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [39]:
# okay, so, yes, I could actually find that element, scroll to it and click 
# it, but altogether this seems so clunky and unelegant if I can just find 
# the element, extract the href and move directly there ... right? No need for
#  scrolling at all

driver.get(driver.find_element_by_partial_link_text('Next').get_attribute(
  'href'))

### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [43]:
url_list = []
there_is_yet_more_data_to_scrape = True

while there_is_yet_more_data_to_scrape:
  print('Scrapy scrapy ...')

  results_table = driver.find_element_by_css_selector(
    'table[summary="Search Results '
    'Table"]')
  rows = results_table.find_elements_by_css_selector(
    'tr.searchfieldtitle')

  # go through each row and collect stuff
  for row in rows:
    name = row.find_element_by_css_selector('td.searchlistitem > '
                                            'span.copybold').text

    try:
      link = row.find_element_by_css_selector('td.searchlistitem + td > '
                                              'a').get_attribute('href')
    except:
      link = ''
  
    data = {
      'name': name,
      'url': link
    }

    url_list.append(data)

  # is there more data to scrape left?
  try:
    driver.get(driver.find_element_by_partial_link_text('Next').get_attribute(
      'href'))
  except:
    there_is_yet_more_data_to_scrape = False

len(url_list)


Scrapy scrapy ...


Scrapy scrapy ...


Scrapy scrapy ...


Scrapy scrapy ...


Scrapy scrapy ...


Scrapy scrapy ...


Scrapy scrapy ...


34

### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [8]:
# yeah ... I guess I already incorporated that before, as well?

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [44]:
df = pd.DataFrame(url_list)
df

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,
4,VAPE FROG,
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
8,VAPE HAVEN,
9,VAPORS LOUNGE,


In [45]:
df.to_csv('08-homework/vape-shops-basic.csv', index=False)

# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [47]:
df['url'][0]

'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ty00ak8Tdnk%3D'

### Use Selenium to visit that page

In [48]:
driver.get(df['url'][0])

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [49]:
data = {
  'mailing_address': driver.find_element_by_xpath('/html/body/table['
                                                  '2]/tbody/tr[4]/td['
                                                  '2]/table[1]/tbody/tr['
                                                  '3]/td[1]').text,
  'location_address': driver.find_element_by_xpath('/html/body/table['
                                                   '2]/tbody/tr[4]/td['
                                                   '2]/table[1]/tbody/tr['
                                                   '5]/td').text,
  'licence_information': driver.find_element_by_xpath('/html/body/table['
                                                      '2]/tbody/tr[4]/td['
                                                      '2]/table['
                                                      '1]/tbody/tr[3]/td['
                                                      '2]').text,
  'total_amount_paid': driver.find_element_by_xpath('/html/body/table['
                                                    '2]/tbody/tr[4]/td['
                                                    '2]/table[2]/tbody/tr['
                                                    'last()]/td[last('
                                                    ')]').text,
  'issued_by': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr['
                                            '4]/td[2]/table[3]/tbody/tr['
                                            '2]/td').text
}
print(data)

{'mailing_address': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804', 'location_address': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801', 'licence_information': 'License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury', 'total_amount_paid': '$ 41.00', 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}


### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [8]:
# this copy-paste job is left to the imagination of the reader

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [58]:

def scrape_maryland_business_details(row):
  try:
    driver.get(row['url'])

    scraped_data = {
      'mailing_address': driver.find_element_by_xpath('/html/body/table['
                                                      '2]/tbody/tr[4]/td['
                                                      '2]/table[1]/tbody/tr['
                                                      '3]/td[1]').text,
      'location_address': driver.find_element_by_xpath('/html/body/table['
                                                       '2]/tbody/tr[4]/td['
                                                       '2]/table[1]/tbody/tr['
                                                       '5]/td').text,
      'licence_information': driver.find_element_by_xpath('/html/body/table['
                                                          '2]/tbody/tr[4]/td['
                                                          '2]/table['
                                                          '1]/tbody/tr[3]/td['
                                                          '2]').text,
      'total_amount_paid': driver.find_element_by_xpath('/html/body/table['
                                                        '2]/tbody/tr[4]/td['
                                                        '2]/table[2]/tbody/tr['
                                                        'last()]/td[last('
                                                        ')]').text,
      'issued_by': driver.find_element_by_xpath('/html/body/table[2]/tbody/tr['
                                                '4]/td[2]/table[3]/tbody/tr['
                                                '2]/td').text
    }

    return pd.Series(scraped_data)
  except:
    print('sorry, no cookie for you')


### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [ ]:
scraped_df = df.apply(scrape_maryland_business_details, axis=1)


,name,url,mailing_address,location_address,licence_information,total_amount_paid,issued_by
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, M...",VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALIS...,License Status: Issued\nLicense No.: 22375606\...,$ 41.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. ..."
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, M...",VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\...,License Status: Issued\nLicense No.: 22375605\...,$ 24.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. ..."
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...,ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\...,VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILL...,License Status: Issued\nLicense No.: 02304705\...,$ 94.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8..."
3,VAPE FROG,,None,None,None,None,None
4,VAPE FROG,,None,None,None,None,None
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...,DISBROW II EMERSON HARRINGTON,VAPE LOFT THE\n185 MITCHELLS CHANCE RD\nEDGEWA...,License Status: Issued\nLicense No.: 02310799\...,$ 154.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8..."
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...,DISCOUNT TOBACCO ESSEX LLC,"VAPE N CIGAR\n7104 MINSTREL UNIT #7\nCOLUMBIA,...",License Status: Issued\nLicense No.: 13343011\...,$ 84.00,"WAYNE A. ROBEY, CLERK OF CIRCUIT COURT\n9250 B..."
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...,FAIRGROUND VILLAGE LLC,VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15...,License Status: Issued\nLicense No.: 06327188\...,$ 179.00,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n..."
8,VAPE HAVEN,,None,None,None,None,None
9,VAPORS LOUNGE,,None,None,None,None,None


In [63]:

complete_df = df.join(scraped_df)
complete_df


,name,url,mailing_address,location_address,licence_information,total_amount_paid,issued_by
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, M...",VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALIS...,License Status: Issued\nLicense No.: 22375606\...,$ 41.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. ..."
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, M...",VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\...,License Status: Issued\nLicense No.: 22375605\...,$ 24.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. ..."
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...,ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\...,VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILL...,License Status: Issued\nLicense No.: 02304705\...,$ 94.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8..."
3,VAPE FROG,,None,None,None,None,None
4,VAPE FROG,,None,None,None,None,None
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...,DISBROW II EMERSON HARRINGTON,VAPE LOFT THE\n185 MITCHELLS CHANCE RD\nEDGEWA...,License Status: Issued\nLicense No.: 02310799\...,$ 154.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8..."
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...,DISCOUNT TOBACCO ESSEX LLC,"VAPE N CIGAR\n7104 MINSTREL UNIT #7\nCOLUMBIA,...",License Status: Issued\nLicense No.: 13343011\...,$ 84.00,"WAYNE A. ROBEY, CLERK OF CIRCUIT COURT\n9250 B..."
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...,FAIRGROUND VILLAGE LLC,VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15...,License Status: Issued\nLicense No.: 06327188\...,$ 179.00,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n..."
8,VAPE HAVEN,,None,None,None,None,None
9,VAPORS LOUNGE,,None,None,None,None,None


## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [64]:
complete_df.to_csv('08-homework/vape-total.csv', index=False)